# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**27/04/2020 - C8 S4**

# Tratamiento de datos y Exploración 

La el manejo de la información contenida en los datos es la razón principal de la construcción de modelos y esquemas de análisis sobre ellos. Tal información se ve afectada por la calidad de los datos, que en segunda instancia, determina el rendimiento de los modelos planteados. Esto hace que sea critico asegurar un preprocesado y una buena examinación de los datasets a trabajar.

El contenido de esta cátedra, se centra en las técnicas esenciales para el preprocesado de datos. 

## Análisis de datos exploratorio

El análisis exploratorio de los datos (EDA en inglés) consiste e utilizar técnicas de sumarización o agregación, con el fin de conocer la distribución de los datos, confirmar hipótesis y contrastar información. Existen muchas maneras de explorar los datos, por ejemplo, se pueden generan visualizaciones, descripciones del conjunto de datos, se pueden también generar agrupaciones y obtener patrones de tales agrupaciones. 

Un concepto recurrente en el análisis exploratorio de datos consiste en el *perfilamiento* de datos. Este hace referencia a la sumarización por medio de estadística descriptiva, aquí existe una variedad de herramientas que pueden ayudar a comprender mejor los datos disponibles. La meta del perfilamiento de datos consiste en generar respuestas y conocimiento en torno al fenómeno que los datos reflejan. En función de los perfiles generados, se puede tener una idea de la calidad del dataset con lo cual es posible decidir como transformar las variables a disposición. A continuación se da una guía a seguir al momento de explorar los datos

### Perfilamiento Univariado

El punto inicial para comprender la naturaleza de una variable, pasa por caracterizar la forma de su distribución, un histograma permite obtener ideas sobre tal faceta.

**Ejemplo**

Se cargan las librerías iniciales

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy import stats

Se carga el dataset de 'house pricing', este consiste en 80 variables (79 variables explicativas más una variable objetivo), describiendo aspectos fundamentales de hogares residenciales en la ciudad de Ames, Iowa. Este dataset está centrado en la regresión sobre el precio final de cada hogar. A continuación se procede a explorar tal dataset.

In [ ]:
# El conjunto a trabajar es el de entrenamiento
df = pd.read_csv('data/train.csv', index_col = 'Id')
df.head()

La cantidad de observaciones corresponde a 1460, por otra parte, posee 79 variables explicativas más un índice.

In [ ]:
df.shape

Se estudia el tipo de valor y cantidad de información faltante para cada columna

In [ ]:
df.info()

Observado lo anterior y la estructura de la base, se aprecia que los datos de tipo `object` hacen referencia a 'strings' o categorias del dataset. Se crea una lista con aquellas columnas.

In [ ]:
object_type_set = [col for col in df.columns if df[col].dtype == 'O']

Se observa la estructura del dataset en tales columnas, en este caso se decide transformarlas a formato 'str' para obtener visualizaciones sobre sus valores. El proceso de transformar los tipos de datos en un dataframe se conoce como *typecasting*. 

In [ ]:
# Se transoforman las columnas anteriores a 'str'
df = df.astype({col:'str' for col in object_type_set})

Sumado a lo anterior, se agregan niveles de multi indexado a las columnas para indicar si son del tipo numérico o categórico. 

In [ ]:
df.head()

In [ ]:
names = ['numeric', 'categorical']

# Se crea una lista con las columnas numericas
numeric = [
    'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
    'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'MiscVal', 'GarageYrBlt',
    'MSSubClass','Fireplaces','SalePrice'
]

# Se crea una lista con las columnas categoricas
categorical = list(set(df.columns) - set(numeric))
''' 
Se generan mappings para el multi indexado del tipo 

[('numeric', col_if_numeric), ...,('categorical', col_if_categorical),...]
'''

mapping = [('numeric', col) for col in numeric]
mapping.extend([('categorical', col) for col in categorical])
'''
Se reordenan las columnas del dataframe para que coincidan con el esquema 
del multi indice
'''

df = df.reindex(columns=numeric + categorical)

Finalemente se asocia el multi indexado

In [ ]:
# Se reasignan las columnas
df.columns = pd.MultiIndex.from_tuples(mapping)

Como observación, se agrega que la columna 'MSSubClass' se clasifica como categórica pues representa un tipo de sector asociado a la propiedad. 

A continuación, se genera una visualización para entender la geometría de cada distribución, en el caso de las variables continuas, se calcula un estimado de la distribución por medio de `kernel density estimation`, este procedimiento consiste en elegir un tipo de función base (en este caso, una gaussiana con media en cada punto y de varianza constante) posteriormente, se calcula el promedio de las funciones base y se obtiene una función representante de la distribución denominada como 'density estimator', esto se hace por medio del método `sns.displot` de la librería `seaborn`. 

In [ ]:
# Grilla de subplots
fig, ax = plt.subplots(nrows=6, ncols=4, figsize=[17, 17])

# Se remueven el ultimo plot
list(map(lambda a : a.remove(), ax[-1,-1:]))

# Se ajusta el espaciado exterior de la figura
fig.tight_layout()

# Se define un titulo y su ubicacion
fig.suptitle('Distribuciones Univariadas Numéricas',
             fontsize=20,
             x=0.5,
             y=1.05)
'''
Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
distinto en funcion del tipo de dato.

'''
for axis, col in zip(ax.flatten(), numeric):
    try :
        # Graficos para datos numericos
        sns.distplot(df[('numeric', col)], ax=axis, rug=True)
               
    except RuntimeError:
        sns.distplot(df[('numeric', col)], ax=axis, rug=True, kde=False)
    
    axis.set_xlabel(col, fontsize=15)

# Se ajusta el espaciado interno entre subplots
w, h = (.4, .4)
plt.subplots_adjust(wspace=w, hspace=h)

Para las variables categóricas, se genera un conteo de valores únicos. Dado que se buscan las distribuciones de forma visual, se elimina información referente a las escalas, que dada la cantidad de gráficos a obtener, solo entorpece el análisis.

In [ ]:
# Grilla de subplots
fig, ax = plt.subplots(nrows=10, ncols=6, figsize=[17, 17])

# Se remueven los ultimos 3 plots
list(map(lambda a : a.remove(), ax[-1,-3:]))

# Se ajusta el espaciado exterior de la figura
fig.tight_layout()

# Se define un titulo y su ubicacion
fig.suptitle('Distribuciones Univariadas Categóricas',
             fontsize=20,
             x=0.5,
             y=1.05)
'''
Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
distinto en funcion del tipo de dato.

'''
for axis, col in zip(ax.flatten(), categorical):

    # Graficos para datos tipos str
    sns.countplot(df[('categorical',col)], ax=axis)
    axis.set_axis_off()
    axis.set_title(col, fontsize=15)
  
    
# Se ajusta el espaciado interno entre subplots
h, w = (.4, .1)
plt.subplots_adjust(wspace=w, hspace=h)

Al observar las distribuciones, es importante buscar si existe variabilidad dentro de estas, pues por lo general, una variable con un único valor casi seguro, no aporta información a la dinámica de los datos.

**Ejemplo**

Se observa la variable 'Heating' (categórica) y se compara con la variable de interés 'SalePrice'. Para ello se usa un gráfico de categórias tipo violín

In [ ]:
# Sirve para fija el tamaño de lasetiquetas del plot
fontdict = {'fontsize':20}

# Estrucutra de figura y axes
fig, ax = plt.subplots(2,1,figsize=[12,13])

# violin plot --> equivalente a catplot(kind = 'violin')

sns.violinplot(('categorical', 'OverallQual'),
            y=('numeric', 'SalePrice'),
            data=df,
            kind='violin',
            ax=ax[0])

sns.countplot(df[('categorical','OverallQual')], ax=ax[1])

ax[0].set_xlabel('OverallQual', fontdict)
ax[1].set_xlabel('OverallQual', fontdict)

ax[0].set_ylabel('SalePrice', fontdict)
ax[0].set_title('Violin plot OverallQuall vs SalePrice', fontdict)
ax[1].set_title('Frecuencias OverallQuall', fontdict)

h, w = (.3, .1)
plt.subplots_adjust(wspace=w, hspace=h)

Un gráfico de violín permite sumarizar y observar características de un dataset. Este se comporta como un gráfico de cajas (boxplot), mostrando la mediana, el rango intercuantílico IQR (percentil 75 - percentil 25, o Q3 - Q1) y el rango 1.5 intercuantílico (Q3 +- 1.5 IQR). Además de lo anterior, se suma una estimación de la densidad por kernel a cada lado. Esto quiere decir, que zonas con mayor densidad, se verán como 'montes' horizontales. 

En el caso de 'OverallQuall', se ve una clara relación entre los distintos niveles de está variable en contraste con difierentes distribuciones de 'SalePrice'. Junto con una distribción que presenta variabilidad, se podría considerar como una de interés. 


Por otra parte, analizando las gráficas univariadas, se puede observar que para 'LandSlope', se tiene poca variablidad y no genera diferencias en distribución para 'SalePrice' en ninguna de sus categorias. 

In [ ]:
# Sirve para fija el tamaño de lasetiquetas del plot
fontdict = {'fontsize':20}

# Estrucutra de figura y axes
fig, ax = plt.subplots(2,1,figsize=[12,13])

# violin plot --> equivalente a catplot(kind = 'violin')

sns.violinplot(('categorical', 'LandSlope'),
            y=('numeric', 'SalePrice'),
            data=df,
            kind='violin',
            ax=ax[0])

sns.countplot(df[('categorical','LandSlope')], ax=ax[1])

ax[0].set_xlabel('LandSlope', fontdict)
ax[1].set_xlabel('LandSlope', fontdict)

ax[0].set_ylabel('SalePrice', fontdict)
ax[0].set_title('Violin plot LandSlope vs SalePrice', fontdict)
ax[1].set_title('Frecuencias LandSlope', fontdict)

h, w = (.3, .1)
plt.subplots_adjust(wspace=w, hspace=h)

**Ejercicios**

1. Los gráficos generados anteriormente siguen exactamente el mismo patrón de generación, lo único que cambia es la columna a analizar. Esto es una mala práctica pues siempre se debe buscar reutilizar código o 'no repetirse' esto se conoce como principio DRY (don't repeat yourself). Construya una función que permita visualizar columnas categóricas del dataset y compararlas con 'SalePrice'. 

2. En función de las visualizaciones construidas, discuta que variables categóricas pueden ser de interés para predecir 'SalePrice'. Busque variabilidad y separación en la distribución de precios. ¿Qué ocurre si una variable categórica posee poca variablidad pero genera buenas separaciones en  'SalePrice'?

Para comparar las variables numéricas, se pueden utilizar gráficos de dispersión contra 'SalePrice'. En este caso, se buscan variabilidad en el histograma univariado y a la vez, se buscan relaciones funcionales (del tipo lineal, exponencial, cuadrático, etc..) con 'SalePrice'. 

**Ejemplo**

Debido a su distribución, se estudia la variable 'GrLivArea', en este caso se define una función para gráficar variables numéricas.

In [ ]:
def scatter_dists(col, df=df, h=.3, w=.1, fontdict={'fontsize': 20}, reg=True):
    ''' Recibe una columna numerica y genera una visualizacion comparativa.
    
    Genera una figura por sobre el dataframe HousePricing (por defecto), recibe 
    parametros extra como el espaciado entre subfigura.
    
    Args:
    ----------
    
    col: String
         El nombre de la columna numerica a visualizar
    
    h,w: float
        Espaciado entre subplot h -> vertical, w -> horizontal
    
    fontdict: dict
             Permite configurar las fuentes de los subplots
    reg: bool
         Permite graficar una regresion lineal sobre los datos (if True)
        
    Returns: None
        Se muestra una figura en pantalla    
    
    '''

    # Estrucutra de figura y axes
    fig, ax = plt.subplots(2, 1, figsize=[12, 13])

    # violin plot --> equivalente a catplot(kind = 'violin')

    if reg:
        sns.regplot(x=df[('numeric', col)],
                    y=df[('numeric', 'SalePrice')],
                    ax=ax[0])
        ax[0].set_title('Regplot plot {} vs SalePrice'.format(col), fontdict)
    else:
        sns.scatterplot(('numeric', col),
                        y=('numeric', 'SalePrice'),
                        data=df,
                        ax=ax[0])
        ax[0].set_title('Scatter plot {} vs SalePrice'.format(col), fontdict)

    
    # Distribucion univariada
    sns.distplot(df[('numeric', col)], ax=ax[1])

    ax[0].set_xlabel(col, fontdict)
    ax[1].set_xlabel(col, fontdict)

    ax[0].set_ylabel('SalePrice', fontdict)
    ax[1].set_title('Frecuencias {}'.format(col), fontdict)

    plt.subplots_adjust(wspace=w, hspace=h)
    

In [ ]:
scatter_dists('GrLivArea')

En este caso, se puede observar una distrbución univariada bien definida y un comportamiento lineal aunque ruidoso. Esto hace que 'GrLivArea' sea una variable de interés. De la misma manera, '1stFlrSF', parece reflejar las mismas buenas características. 



In [ ]:
scatter_dists('1stFlrSF') 

En el caso de 'TotalBsmtSF' se tiene

In [ ]:
scatter_dists('TotalBsmtSF')

Una relación menos lineal con un poco más de ruido pero una buena distribución en e dataset. Esta variable puede ser de interés pero esto se puede estudiar a posteriori. 

Finalmente para 'MasVnrArea', se tiene

In [ ]:
scatter_dists('MasVnrArea', reg = False)

Se aprecia una distribución altamente concentrada y poco relacionada con la variable a predecir, a priori, se puede considerar como una variable de poco interés en el análisis. 

**Ejercicio**

1. Estudie las siguientes proposiciones: 

    1.'OverallQual' y 'YearBuilt' parecen relacionadas con 'SalePrice'. 
    2. En el caso de 'OverallQual', esta relación es bastante débil.
    3. En el caso de 'YearBuilt', esta relación es bastante débil.
    4. Los gráficos de caja para 'OverallQual contra  'SalePrice' muestran cierta linealidad con respecto a 'SalePrice'.

2. Estudie la distribución univariada de 'SalePrice', a continuación ejecute el test K^2 de D’Agostino usando `normaltest` del módulo `stats` de SciPy. Compare para una significancia de 5%. ¿ Se puede tratar esta variable como distribuida de manera normal, tomando en cuenta su comportamiento estadístico?

3. Las distribuciones de 'TotalBsmtSF' y '1stFlrSF' parecen bastante similares, más aún sus relaciones con 'SalePrice' comparten una tendencia. Ejecute el [test de Kolmogorov-Smirnov ](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html) por medio de `ks_2samp` para explorar la hipótesis:

 ```'TotalBsmtSF' y '1stFlrSF' vienen de la misma distribución```
 
4. Estudie algunos estadísticos de interés según el tipo de dato.
    
    1. Para las variables numéricas estudie promedios, desviaciones estándar y rangos intercuartílicos. Utilice los rangos calculados para tener una idea del porcentaje de valores fuera de tales rangos por columna. 
    
    2. Para variables catégoricas calcule frecuencias, proporciones y modas. Utilice lo anterior para obtener alguna idea de la variabilidad de los datos.
    

### Perfilamiento Bivariado

Basándose en el perfilamiento anterior, es de utilidad observar relaciones entre variables de interés. Para esto se pueden emplear visualizaciones a pares. 

**Ejemplo**

Se selecciona un conjunto de variables de interés y se investigan sus relaciones bivariadas.

In [ ]:
# Se genera una función auxiliar

def indexer(cols, t_c = df.columns):
    '''Genera columnas multinivel a partir de nombres de columna planos.'''
    
    set_to_tuple = set(*[cols])

    tuples = [
        i for i in t_c if set_to_tuple.intersection(set(i))
    ]
    
    return tuples

Se selecciona un conjunto de variables a examinar

In [ ]:
interest = [
    'SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
    'FullBath', 'YearBuilt'
]

idxs = indexer(interest)

In [ ]:
df[idxs].head()

Se procede a observar el comportamiento bivariado de las columnas seleccionadas

In [ ]:
# Pone SalePrice al final de la lista
idxs.sort()
idxs.remove(('numeric', 'SalePrice'))
idxs.append(('numeric', 'SalePrice'))

In [ ]:
'''
Seaborn presenta problemas para multi indices en columnas, se 
procede a eliminar el nivel exterior y a obtener la visualización
correspondiente.
'''
data = df.reindex(idxs, axis=1).droplevel(0,axis=1)
sns.pairplot(data = data, diag_kind='kde')

La última fila de la visualización anterior entrega una idea de la relación entre 'SalePrice' y las demás variables de interés. Dentro de estas relaciones, se observan ciertos comportamientos lineales y en particular para 'OveralQuall' y 'YearBuilt' se observa cierta exponencialidad. Dentro de las interacciones entre variables, se observa que 'GrLivArea' y 'TotalBsmtSf' se comportan de manera similar contra 'OverllQuall', esperandose cierta tendencia creciente en ambos casos.

Los análsis iniciales basados en visualizaciones sirven para comprender a grandes rasgos la estructura del dataset. Este tipo de exploración debe ser acompañada de tests estadísticos como los vistos en los ejercicios anteriores. En el caso del perfilamiento bivariado se puede usar una técnica mixta, basada en el análisis de las correlaciones.

**Ejemplo**

Se construye una matriz de correlaciones y se visualiza para todo el dataset.

In [ ]:
corrmat = df.corr()

se muestran las dos variables más correlacionadas (positivamente) con 'SalePrice'.

In [ ]:
col = indexer(['SalePrice'])
corrmat[col].nlargest(3,col)

En cuanto a correlación negativa, no se ven relaciones lineales inversas de mayor fortaleza 

In [ ]:
corrmat[col].nsmallest(3,col)

Se procede a visualizar

In [ ]:
'''
Se inserta 'SalePrice' como primera fila x columna de la matriz de correlacion
'''

unsorted = list(corrmat.columns)
unsorted.remove(*col)
unsorted.insert(0, *col)

sortd = pd.MultiIndex.from_tuples(unsorted)
corrmat = corrmat.reindex(index = sortd, columns = sortd)
'''
Dado lo anterior, se ajusta el anchor de colores con maximo en .9
y -0.5, para tener una perspectiva entorno a los valores maximos 
de correlacion (negativa y positiva)
'''

fig, ax = plt.subplots(figsize=[16, 14])

sns.heatmap(corrmat, vmin=-.5, vmax=.9, linewidths=.01)

Según el esquema de valores, se buscan los puntos más claros y más oscuros fuera de la diagonal. En primera instancia, las variables  'TotalBsmtSF' y '1stFlrSF' parece bastante correlacionadas, lo mismo ocurre con la variables 'GarageCars' y 'GarageArea', esto puede indicar multicolinearidad que implica información duplicada o relacionada de manera trivial en el dataset. 

Las correlaciones con 'SalePrice' deben ser analizadas con más detenimiento, aquí se ve que  'GrLivArea', 'TotalBsmtSF', y 'OverallQual' juegan un papel preponderante.

**Ejercicio**

1. Obtenga las 15 correlaciones más altas (positiva o negativa) con 'SalePrice'. Reindexe la matriz de correlaciones, de manera tal que contenga 1's en la diagonal y 'SalePrice' sea la primera fila - columna. 

2. Muestre los coeficiente de correlación dentro de cada casilla del gráfico de correlaciones. Utilice esa información en conjunción con el perfilamiento univariado para filtrar variables de interés.

Las correlaciones pueden ser interpretadas con datos mixtos pero se recomienda analizar sus valores cuando se trabaja con valores continuos (comparación variable continua vs continua). Para analizar valores categóricos (categórico vs categórico) existen herramientas especializadas una de ellas es por medio de tablas de dos tratamientos o de contingencia (2 way tables). 

**Ejemplo**

Se construye una tabla para analizar 'OverallQual' vs 'GarageCars'

In [ ]:
to_compare =['OverallQual','GarageCars']
data_cat = df['categorical']

kwargs = {'index': data_cat[to_compare[0]], 'columns': data_cat[to_compare[1]]}

# Se construye la tabla
tabla = pd.crosstab(**kwargs, margins=True, margins_name='Total')
tabla

En el caso anterior, la función `pd.crosstab(**kwargs, margins=True, margins_name='Total')` es equivalente a

```python
data_cat.pivot_table(**kwargs, values = 'OverallQual',aggfunc='count', fill_value=0)
```
y permite calcular el numero de ocurrencias de una variable para cada una de sus categorías en comparación con los valores de otra variable. Se añaden los totales como margenes de la tabla. En este caso, podemos deducir las interacciones entre las variables, de manera similar como actúa la correlación en variables continuas. 

Para el caso de  'OverallQual' y 'GarageCars' vemos que tienden a acumularse dentro de una rango reducido, se puede concluir que a medida que 'OverallQual' crece entre 4 y 6, aparece un aumento considerable en la categoría 'GarageCars' hasta que esta última llega al valor 2, valores superiores paracieran ser independientes de 'OverallQual'. 

**Ejercicios**

1. Compare variables categóricas usando este método, ¿ se puede encontrar alguna relación entre categórias?

2. Es posible aplicar este método para comparar variables categóricas y continuas, para esto se necesita categorizar la variable continua objetivo. Categoríce la variable 'SalePrice' en 5 tramos y compare con 'OverallQual' ¿Se observa alguna tendencia?


Otra forma de comparar variables categóricas es por medio de un test $\chi^2$. Este permite obtener un indicador de significancia estadística entre variables. Se basa en una tabla de contingencia y proporciona la probabilidad de que dos variables categóricas sean independientes basádandose en el estádistico $\chi^2$, entrega también un arreglo con frecuencias esperadas.

In [ ]:
from scipy.stats import chi2_contingency

# Se debe trabajar la tabla sin margenes
tabla = pd.crosstab(**kwargs, margins=False)

chi2, p, dof, ex =chi2_contingency(tabla)

La tabla de frecuencias esperadas se puede interpretar de la siguiente forma:

In [ ]:
expected_freq = pd.DataFrame(ex, index=range(1,11))
expected_freq.index.name = 'OverallQual'
expected_freq.columns.name = 'GarageCars'
expected_freq

Aquí, la frecuencia esperada para la la categoría 1 'OverallQual' de estar en la categoría 0 de 'GarageCars' es 0.11. Se puede decir que esta configuración es muy poco probable en comparación a otras como pertenecer a la categoría 6 de 'OverallQual' y 2 de 'GarageCars'. Este tipo de tablas permite clasificar las relaciones entre variables categóricas y obtener *insights* sobre las dinámicas que el dataset refleja. 

El valor $p$ entregado por el cáculo corresponde a:

In [ ]:
p 

Si para este test llamamos $\alpha$ al valor de significancia, se puede resumir:

1. Si $p > \alpha$ no hay evidencia para rechazar la hipótesis nula por lo que se pueden considerar independientes.

2. Si $p \leq \alpha$ hay evidencia para rechazar la hipótesis nula por lo que se puede decir que existe una dependencia estadística entre las variables. 


Para una significancia del 5% , hay evidencia para rechazar la hipótesis de independencia entre 'OverallQual' y 'GarageCars', luego puede existir un factor latente que las relaciona (¿será 'SalePrice'?)

In [ ]:
p <= 0.05

Como consideración general, para que este test sea consistente estadísticamente, se deben observar frecuencias (esperadas y observadas) mayores a 5. 

**Ejercicio**

1. Utilice el gráfico de correlaciones para escojer dos variables categóricas de interés. Verifique si existen relaciones estadísticas entre ellas. 

Finalmente, para comparar variables numéricas y categóricas, es posible utilizar técnicas especializadas como lo son los tests Z y T. Estos tests se utilizan de manera simultanea con gráficos de caja (o violín), donde cada caja representa una categoría. 

Tanto el test Z como el T permiten verificar si las medias de dos grupos son estadísticamente diferentes entre si. aquí, el estadístico Z se define por

\begin{equation}z=\frac{\left|\bar{x}_{1}-\bar{x}_{2}\right|}{\sqrt{\frac{s_{1}^{2}}{n_{1}}+\frac{s_{2}^{2}}{n_{2}}}}\end{equation}

Si su probabilidad asociada es pequeña, entonces **la diferencia de las medias** es significativa. 

Por otra parte, el estadístico T es más robusto a tamaños de observaciones pequeños (menores que 30 por ejemplo), este viene dado por 

\begin{equation}
t=\frac{\bar{X}_{1}-\bar{X}_{2}}{\sqrt{S^{2}\left(\frac{1}{N_{1}}+\frac{1}{N_{2}}\right)}}
\end{equation}

Donde 

\begin{equation}
S^{2}=\frac{\left(N_{1}-1\right) S_{1}^{2}+\left(N_{2}-1\right) S_{2}^{2}}{N_{1}+N_{2}-2}
\end{equation}

Aquí , $\bar{X}_{1}, \bar{X}_{2}$ son las medias, $S_{1}^{2}, S_{2}^{2}$ varianzas y $N_1$ , $N_2$ los totales de cada grupo a testear. 

**Ejercicios**

1. Utilice el test de independencia $t$ (o 2 - sample $t$-test) para comparar 2 variables continuas de interés. [*Hint*](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)

2. Observe que el caso categórico vs continuo, cada categoría representa un grupo de valores continuos asociados. Por ejemplo, si la variable categórica `A` tiene las categorías `c_1` y `c_2`, al compararla con la variable continua `B`, es necesario agrupar los valores de `B` para `c_1` y para `c_2` para luego estudiar su independencia. Utilice el test anterior para medir independencias de grupos entre una variable categórica vs 'SalePrice'. **Obs**: La variable categórica debe ser bivariada.

3. Utilice el test Z con las variables anteriores y compare. ¿Qué restricciones extra posee este test?

Finalmente, se puede hacer uso de un test F o ANOVA. Este test permite comparar más de una media al mismo tiempo, una manera simple de aplicar este test consiste en método conocido como **one way ANOVA**, aquí, se testea si más de 2 grupos son similares basados en sus medias. En este caso, la hipótesis nula es 

`No hay diferencia significativa entre los grupos`

**Ejemplo**

Se selecciona la variable 'GarageCars' y se compara con 'SalePrice'. 

In [ ]:
idx = indexer(['SalePrice','GarageCars'])
grouped = df[idx].groupby(idx[1])

En la variable 'GarageCars' se distinguen 5 categorías

In [ ]:
len(df[idx[1]].unique())

a partir de la agrupación anterior, se forman entonces 5 grupos de valores para 'SalePrice'.

In [ ]:
len(grouped.groups)

Se obtienen los grupos

In [ ]:
total_groups = len(grouped.groups)
groups = [grouped.get_group(i) for i in range(total_groups)]

Se muestra el grupo correspondiente a la categoría 0

In [ ]:
groups[0].head()

Se limpia el formato de cada grupo

In [ ]:
def group_cleaner(group):
    ''' Limpia un grupo.
    Reconoce la categoria del grupo, en la posicion [:,1], 
    guarda ese nombre y elimina la columna de categoria, 
    posteriormente renombra la columna.
    
    Args:
    ----------
    
    group: pandas Groupby object
          Recibe una agrupacion para categorias
          
    Returns:
    ----------
        pandas Grppuby object
        Entrega el grupo ordenado.
    '''
    group_0 = group.copy()
    name = group_0.iloc[0,1]
    group_0.drop(indexer(['GarageCars']), axis=1, inplace=True)
    group_0.columns  = ('cat_{}'.format(name),)
    
    return group_0

se procede a limpiar

In [ ]:
groups_to_test = list(map(group_cleaner, groups))

Se muestra el grupo correspondiente a la categoria 0 post limpieza

In [ ]:
groups_to_test[0].head()

Se procede a testear

In [ ]:
from scipy.stats import f_oneway

F,p = f_oneway(*groups_to_test)

print('Estadistico F:',F)
print('p valor :', p)

probando para una significancia del 5% se tiene hay evidencia para rechazar la hipótesis nula y por tanto hay una diferencia significativa entre los grupos. 

In [ ]:
alpha = 0.05
p <= alpha

**Ejercicio**

1. Compruebe el resultado del test ANOVA anterior con un analísis visual por medio de gráficos de violín.

## El problema de los datos faltantes 

Los métodos estándar de manejo de datos han sido desarrollados para para analizar arreglos tabulares. Por lo general las filas de tal arreglo representan observaciones y las columnas sus características asociadas. Cada entrada en este arreglo puede ser modelada como un número, siendo este ligado a un proceso subyacente continuo o discreto. Para comprender tal proceso, es de utilidad sumarizar y observar los valores faltantes con el fin de obtener patrones y seleccionar estrategias para tratarlos. 

### Exploración de valores faltantes

Cuando los datos faltantes se encuentran en variables que no son de interés, se pueden obviar y pasar a trabajar directamente en ingeniería de *features* e implementación de modelos de aprendizaje automático, por tal motivo, el análisis exploratorio y visualización se considera como primer paso en un procedimiento de análsis de datos. Sin embargo, una exploración preliminar de valores faltantes puede ser útil en conjunto con los perfilamientos visuales y estadísticos realizados.

**Ejemplo**

Observe que en los perfilamientos anteriores, las variables categóricas:

```python
var_missing = ['GarageQual', 'GarageCond', 'BsmtFinType1','BsmtCond', 'GarageFinish', 'Fence', 'BsmtExposure',  'BsmtQual', 'MiscFeature', 'GarageType', 'Electrical', 'FireplaceQu', 'BsmtFinType2','MasVnrType']
```
Parecen no tener valores faltantes.

In [ ]:
var_missing = [
    'GarageQual', 'GarageCond', 'BsmtFinType1', 'BsmtCond', 'GarageFinish',
    'Fence', 'BsmtExposure', 'BsmtQual', 'MiscFeature', 'GarageType',
    'Electrical', 'FireplaceQu', 'BsmtFinType2', 'MasVnrType'
]

var_missing = indexer(var_missing)

df[var_missing].isnull().sum()

Sin embargo, basta observar las columnas para comprender que tales variables si poseen valores faltantes

In [ ]:
df[var_missing].head()

Por tal motivo es necesario realizar una exploración inicial de los datos faltantes en conjunción con los análisis de distribución iniciales. 

**Ejercicios**

1. Estudie la distribución de los valores faltantes en las variables numéricas.

2. Considerando que para las variables categóricas las variables con valor 'nan' son consideradas como una nueva categoría. ¿Se ven afectados los análisis anteriores sobre sus distribuciones?

**Ejemplo**

Para estudiar en mayor profundidad la distribución de los valores faltantes, se procede a transformarlos en formato `np.nan`

In [ ]:
df.replace('nan',np.nan, inplace = True) 

En términos generales, los valores perdidos de este dataset se encuentran relativamente limpios pues están estandarizados con la categoría 'nan'.


Dado que sumarizar valores faltantes genera una estructura de datos, vale la pena explorarla visualmente, para facilitar tal tarea, existe la librería `missingno`

In [ ]:
import missingno as msno

Las visualizaciones de generadas por medio de esta librería pueden ser utilizadas para discutir el problema de valores faltantes y generan una estrategia para su tratamiento.

**Ejemplo**

Se genera una visualización sobre la distribución de valores perdidos en el dataset. En primer lugar, se confirma que la conversión 'nan' $\mapsto$ `np.nan` sea reconocida en el dataset

In [ ]:
df.isnull().sum()

Vemos que en efecto aparecen los valores faltantes antes ignorados. En este apartado se observa que dentro de las variables categóricas se encuntra la mayor cantidad de información perdida. 

In [ ]:
df.isnull().sum().nlargest(10)

mediante la libreria `missingno` es posible ver el panorama completo de los valores faltanes en el dataset de manera sencilla 

In [ ]:
fig, ax = plt.subplots(figsize = [15, 10])
msno.matrix(df,ax = ax, sparkline=False)

**Ejercicio**

1. Genere un subconjunto con las 10 columnas con mayor información faltante y genere el gráfico anterior sin usar un objeto `axes` y con la opción `sparkline=True`.

Esta visualización muestra que exiten columnas practicamente sin información, según la agregación anterior, estas corresponden a 'PoolQC', 'MiscFeature' y 'Alley'. 

Por medio de correlaciones entre valores faltantes, es posible obtener un análisis bivariado análogo al anteriormente generado. Para ello se puede utilizar un mapa de calor.

In [ ]:
fig, ax = plt.subplots(figsize = [15, 10])
msno.heatmap(df, ax = ax)

Este gráfico muestra correlaciones de nulidad entre pares de variables, estas varian desde -1 a 1, donde -1 significa que las variables son excluyentes, es decir, la aparición de una hace que la otra este ausente. Por otra parte el valor 1 corresponde inclusión, esto quiere decir, que la aparición de una hace que la otra aparezca. Valores cercanos a 0 (sin valor numérico en el gráfico) indican ausencia de relación de nulidad entre las variables.

En el gráfico recien generado, no se observen relaciones de nulidad negativa, por otra parte, existen variables fuertemente relacionadas en cuanto a su información como lo son 'MasVnrType' y 'MasVnrArea', el comportamiento general es que la información esta fuertemente relacionada (en el sentido de inclusión de información) o simplemente no lo está. 


**Ejercicio**

1. El gráfico de correlaciones de nulidad permite tener una idea de como se relaciona la información faltante en pares de variables. Para comparar más de dos variables es posible utilizar un *dendograma*. Utilice las 20 variables con mayor cantidad de valores faltanes visualice su dendograma por medio de `msno.dendogram`. Interprete los resultados.[*Hint*](https://github.com/ResidentMario/missingno)




### Una perspectiva teórica

Para dar un contexto teórico al problema de valores faltantes, se define la matriz de datos $Y=\left(y_{i j}\right)$, la cual representa un arreglo rectangular de datos. Para cada elemento de esa matriz, se asocia una variable indicadora, que da lugar a una *matriz indicadora de información faltante* definida por  
$R=\left(r_{i j}\right)$. 

Por simplicidad, se puede asumir que las filas $\left(y_{i}, r_{i}\right)$ son i.i.d sobre $i$. Es posible así, modelar la existencia de un proceso (o fenómeno) que causa la perdida de información sobre $Y$ por medio de una distribución condicional de $R$  dado $Y$, denotada por $\mathcal{P} \left(R \mid Y_{obs}, Y_{mis} ,\phi \right)$, donde $\phi$ denota un conjunto de parámetros que modelan el proceso de perdida de información e $Y_{obs}$, $Y_{mis}$, denotan aquellas entradas de la matriz de datos $Y$ con información observada y faltante respectivamente. Si para tal proceso no se aprecia una relación con los valores faltantes en $Y$, es decir, 

\begin{equation}
\tag{eq:1}
\label{eq:1}
\mathcal{P} \left(R \mid Y_{obs}, Y_{mis} ,\phi \right)= 
\mathcal{P} \left(R \mid \phi \right)
\end{equation}

Se dice entonces que el mecanismo de perdida de información es del tipo *información faltante completamente aleatoria* o MCAR (missing completely at random). Por otra parte, cuando la probabilidad de la información faltante en $Y$ se relaciona con variables observadas, así, $R$ depende de $Y_{obs}$ pero no de $Y_{mis}$, por lo que la distribución pasa a ser


\begin{equation}
\tag{eq:2}
\label{eq:2}
\mathcal{P} \left(R \mid Y_{obs}, Y_{mis} ,\phi \right)=
\mathcal{P} \left(R \mid Y_{obs}, \phi \right)
\end{equation}

Entonces, el proceso de perdida de información se denomina como, *información faltante aleatoria* MAR (missing at random). El proceso es llamado *información faltante no aleatoria* MNAR, si la distribución de $R$ depende de las componentes faltantes de $Y_{mis}$, es decir, la ecuación inicial ($\ref{eq:2}$) no se cumple para algunas filas de $Y$ y algunos valores de las componentes faltantes.

En los métodos discutidos en esta cátedra $R$ e $Y$ serán modelados por medio de distribuciones conjuntas, es decir, son tratadas como variables aleatorias. 

**Ejemplo**

La estructura de datos con información faltante más simple, ocurre en el caso univariado. Acá, $Y$ y $R$ son vectores, luego 

\begin{equation}
\mathcal{P}(Y=y, R=m | \theta, \phi)=\prod_{i=1}^{n} f_{Y}\left(y_{i} | \theta\right) \prod_{i=1}^{n} f_{R \mid Y} \left(r_{i} | y_{i}, \phi\right)
\end{equation}

donde $f_{Y}\left(y_{i} | \theta\right)$ denota la densidad de la componente $i$-sima de $Y$, $y_{i}$, parametrizada por $\theta$, y $ f_{R \mid Y} \left(r_{i} | y_{i}, \phi\right)$ es la densidad de una variable aleatoria Bernoulli para el indicador binario $r_{i}$, con probabilidad $\mathcal{Pr}\left(r_{i}=1 | y_{i}, \phi\right)$ para $y_{i}$ valor faltante. Si el proceso de perdida de información es independiente de $Y$, es decir, $\mathcal{Pr}\left(r_{i} = 1 | y_{i}, \phi\right)=\phi$ , constante que no depende de $y_{i}$, entonces el proceso de perdida de información es MCAR. Si tal mecanismo depende de $y_{i}$, entonces es MNAR pues pasa a depender de los valores perdidios de $y_{i}$.

**Ejercicio**

Si se supone que $Y$ es una variable $n$ dimensional con posibles valores faltantes, $R$ es el indicador de pérdida de información para $Y$ y $X$ es una variable  $n$ dimensional relacionada al mismo dataset pero con valores completamente observados. En este contexto, si el dataset consiste de $n$ observaciones, donde para $r < n$  fijo, se tiene que $i=1, \ldots, r$ $X_i$ e $Y_i$ son observados, mientras que para $j = r+1, \ldots, n$ se tiene $X_j$ observado pero $Y_j$ faltante. 

1. Si para $i = 1, \ldots, n$ se asume que $y_i$ es independiente de $r_i$ dado $x_i$. Aplique el supuesto MAR sobre este conjunto de datos y deduzca una expresión para $\mathcal{P}(R \mid X, Y , \phi)$ que no dependa de $Y$. 

2. Utilice la expresión anterior para deducir que $R$ e $Y$ son independientes dado $X$. 

3. Utilice lo anterior para deducir que la distribución condicional de $Y$ dado $X$ y $R$ no depende de $R$. 

4. Deduzca que la distribución condicional de $Y$ dado $X$ puede ser estimada para componentes con $Y$ observado ($r_i = 0$) para luego ser utilizada para predecir valores faltantes de $Y$ ($r_i = 1$). 

Para comprender el significado de lo anterior, considere el siguiente ejemplo

**Ejemplo**

Se realiza un experimento en cual se examinan ciertas variables relacionadas a la salud, dentro de estas variables, se consideran algunos comportamientos como *beber alcohol*, *fumar*, *consumo de drogas* y *actividad sexual*. El experimento se realiza sobre una población menor de 18 años, luego, por regulaciones gubernamentales, no se pueden realizar preguntas sobre actividad sexual a menores de 14 años. Observe

1. La variable asociada al comportamiento sexual cumple la hipótesis MAR, pues en efecto, al observar la variable de edad (variable observada $Y_{obs}$), se puede comprender la falta de información subyacente.

2. Por otra parte, suponga que la variable de edad y comportamiento sexual están altamente correlacionadas. Asuma además que en el estudio existe un valor de *índice de salud* asociado a cada participante. Los investigadores del experimento deciden utilizar regresión lineal sobre el valor del *índice de salud* para estudiar el peso de cada variable en el modelo. Al hacer esto, eliminan la variable de edad por producir multicolinearidad. 
Si los investigadores relacionan los valores del *índice de salud* con la probabilidad de que la variable sexual tenga valores faltantes, se está en la hipótesis MNAR, pues la variable edad pasa a ser no observada. 

## Interludio: Introducción a Scikit Learn

Dentro la extensa variedad de librerías para el manejo de datos e implementacion de modelos, se encuentra [Scikit-learn](https://scikit-learn.org/stable/). Está librería es estándar en flujos de trabajo con datos y provee herramientas clásicas de aprendizaje automático implemententadas de manera eficiente. Sus APIs permite generar código limpio y está provista de una extensa documentación.


Una gran ventaja de Scikit-learn consiste en su estructura transversal de clases, construida sobre una lista simple de APIs y patrones de diseño. Las Apis más representativas son:

* *transformers*: Permite transformar datos input antes de utilizar algoritmos de aprendizaje sobre ellos. Con esto, se pueden realizar imputaciones de valores faltantes, estandrización de variables, escalamientos y seleccion de caracterísiticas por medio de algoritmos especializados.

* *estimators*: La interfaz de esimadores es uno de los componentes más importantes. Los algoritmos de aprendizaje automático están implementados aquí. El proceso de aprendizaje de tales algoritmos es manejado según la inicialización de un objeto alojado en el módulo, esto consiste proporcionar los hiperparámetros que definen el modelo a entrenar, antes de proporcionar datos. El segundo paso corresponde a utilizar el método `.fit()` sobre los datos a utilizar, aquí se aprenden los parámetros y se encapsulan sus valores como atributos públicos para fácil inspección. 

* *predictors*: Esta interfaz permite generar predicciones usando un estimador previamente entrenado sobre datos (a priori) desconocidos. 

El método usual de importación se basa en seleccionar un submódulo de la librería indicando (de manera opcional) el objeto que se utilizará, por ejemplo, si se desea utilizar el escalador de datos Min-Máx del submódulo `preprocessing`, se haría de la manera usual, por medio de:

```python
from sklearn.preprocessing import StandardScaler
```

**Obs**: No se recomienda importar la librería completa `import sklearn as sk` pues su estructura de submódulos es suficientemente grande, como para considerar cada uno como una librería. 


A lo largo del curso se estudiarán distintos componentes de esta librería, durante la siguiente sección nos centraremos en los móduloz `preprocessing`, `compose` y `pipeline`.

### Preprocesamiento de datos con Scikit-Learn

El módulo `sklearn.preprocessing` entrega funciones de manejo de datos ampliamente utilizadas en la práctica. Hace uso de *transformers* con lo que facilita la transición de datos 'crudos' a un formato estándar para el entrenamiento de algoritmos. 

Dentro de tales algoritmos encontramos:

#### Estandarización

La estándarización es el primer tipo de transformación a tener presente, esto pues, una gran cantidad de algoritmos de aprendizaje automático / estadístico, asumen que los datos a operar se encuentran distribuidos de manera normal. 

En la práctica, se ignora la forma de la distribución a trabjar y simplemente e transforma removiendo la media y escalando por la desviación estándar.

**Obs**:Esto **no** es recomendaddo si el histograma de la variable objetivo dista mucho de ser gaussiana, para tal caso se revisarán transformaciones alternativas.

El objeto `StandarScaler` permite estandarizar datos.

**Ejemplo**

Se generan 3 distribuiones a estandarizar:


In [ ]:
df_0 = pd.DataFrame({
    'x1': np.random.normal(0, 2, 1460),
    'x2': np.random.normal(10, 4, 1460),
    'x3': np.random.normal(-15, 6, 1460)
})
df_0.plot.kde()

Se importa el escalador y se inicializa

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

Se generan datos escalados, el método `.fit_transform` es transversal en los transformadores de `preprocessing`, lo que hace es obtener los parámetros de transformación de los datos proporcionados y transformar (todo en un paso). Es una abreviación de los métodos `.fit` para obtención de parámetros y `.transform()` para aplicar la transformación a nuevos datos. 

Por lo anterior, al aplicar `.fit_transform()` se obtienen los parámetros de media `.mean_` y desviación estándar `.scale_` para cada columna del dataframe operado. Observe que tales atributos del objeto tipo `StandardScaler` son públicos, observe además que es posible crear objetos que hereden de tal clase y por tanto, anular sus métodos utilizando métodos propios, recuerde que Python soporta Duck typing.

Se obtienen los parámetros y se transforman las columnas generadas

In [ ]:
df_1 = scaler.fit_transform(df_0)
df_1 = pd.DataFrame(df_1, columns=['x1','x2','x3'])

In [ ]:
df_1.plot.kde()

A continuación se implementa la función `scaler_test` que simplifica el proceso anterior

In [ ]:
def scaler_test(df, scaler, dat=False):
    ''' Simplifica el proceso de testear transformadores de datos.'''
    
    data = df.copy()
    data = scaler.fit_transform(data)
    data = pd.DataFrame(data, columns=df.columns)
    data.plot.kde()

    if dat:
        return data

#### Escalamiento mínimo-máximo

Una buena alternativa al método anterior, es el escalamiento por rango, este tiene la forma:

\begin{equation}
\frac{x_{i} - \min(x)}{\max (x)-\min (x)}
\end{equation}

para $x$ columna a tratar, $x_i$ elemento a transformar. Esta transformación permite hacer que los datos se muevan entre 0 y 1 y puede ser utilizado y la distribución de los datos a tratar no cumple la hipotesis de normalidad (recordar test Z). Observe que este transformador se ve afectado por la presencia de outliers. 

**Ejemplo**

Se generan 3 distribuciones y se estudia la transformación

In [ ]:
df_0 = pd.DataFrame({
    'x1': np.random.beta(8, 2, 1460)*100,
    'x2': np.random.chisquare(10, 1460),
    'x3': np.random.normal(50, 3, 1460)
})

df_0.plot.kde()

Se importa el objeto `MinMaxScaler`

In [ ]:
from sklearn.preprocessing import MinMaxScaler

Se prueba la transformación,

In [ ]:
scaler = MinMaxScaler()
data = scaler_test(df_0, scaler, dat=True)

Se comprueban minimos y máximos:

In [ ]:
print('data min:', data.min())
print('data max:', data.max())

Si se desea escalar por rengo, la mejor práctica es comprender los mínimos y máximos *absolutos* para cada columna. Esto se refiere, a las cotas superiores e inferiores que posee la columna **por definición**, a modo de ejemplo, considere un dataframe con las notas de una asginatura donde se enzeña análisis de datos, se sabe que la nota máxima en cierto ítem se codifica en una columna y su máximo es en efecto es 7.0, sin embargo el mínimo en dicha columna es 1.5, que es distinto al mínimo natural para dicho item que es 1.0. Esto puede acarrear problemas con datos nuevos, sobretodo si aparece una nota inferior a 1.5. 

**Ejercicios**

1. Investigue los parámetros que se deben usar para proporcionar escalamiento por rango con valores máximos y mínimos proporcionados explícitamente. 

2. Estudie el transformador `MaxAbsScaler`.

#### Transformación robusta

Cuando se trabaja con columnas que poseen valores fuera de rango (outliers) las transformaciones anteriores pueden fallar. En este caso, se recomienda utilizar una transformación de la forma

\begin{equation}
\frac{x_i - Q_1(x)}{IQR(x)}
\end{equation}

Donde $IQR = Q_3(x) - Q_1(x)$ es el rango intercuartílico de la columna $x$. 

**Ejemplo**

Se generan datos con outliers

In [ ]:
df = pd.DataFrame({
    'x1': np.concatenate([np.random.normal(20, 1, 1460), np.random.normal(1, 1, 25)]),
    'x2': np.concatenate([np.random.normal(-10, 1, 1460), np.random.normal(50, 1, 25)]),
})
df.plot.kde()

Se importa el objeto `RobustScaler` y se aplica

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

scaler_test(df, scaler)

Se puede observar como los datos son centrados pero se mantienen los outliers generados.

**Ejercicio**

1. Sea $k(x,y)$ un kernel definido por $\langle \phi(x), \phi(y) \rangle_{\mathcal{H}}$, donde $\phi: \mathcal{X} \rightarrow \mathcal{H}$ es una fución que opera elementos entre espacio de datos $\mathcal{X}$ y un espacio de Hilbert $\mathcal{H}$. Se sabe por el *truco del kernel* que algoritmos kernelizados obvian el tratamiento de *features* no lineales representadas por transformaciones del tipo $\phi(\cdot)$. Sin embargo, es posible centrar datos en el espacio de carácteristicas $\mathcal{H}$ utilizando la matriz de Gramm asociada a $k(x,y)$. Para efectuar tal procedimiento, estudie la clase `KernelCenterer`. ¿Que ventaja tiene usar este método?

#### Mapeo a distribuciones gaussianas 

Como mencionó anteriormente, no siempre se cumple la hipótesis de normalidad en las columnas de un dataset, en tal caso, no es una buena idea estandarizar los datos pues puede llevar a problemas al momento de operar con algoritmos que requieren normalidad en su formulación. Existe una familia de transformaciones paramétrica que busca aproximar una distribución arbitraria a una gaussiana, se accede a este tipo de transformaciones por medio de la clase `PowerTransformer`, en esta clase se encuentran 2 transformaciones:

* Yeo-Johnson dada por: 

\begin{equation}
x_{i}^{(\lambda)}=
\begin{cases}
\left[\left(x_{i} + 1\right)^{\lambda}-1 \right] / \lambda & \text { si } \lambda \neq 0, x_{i} \geq 0 \\
\ln \left(x_{i}+1\right) & \text { si } \lambda=0, x_{i} \geq 0 \\
-\left[\left(-x_{i}+1\right)^{2-\lambda}-1\right] /(2-\lambda) & \text { si } \lambda \neq 2, x_{i}<0 \\
-\ln \left(-x_{i}+1\right) & \text { si } \lambda=2, x_{i}<0
\end{cases}
\end{equation}

* Box-Cox: Solo puede ser utilizada en datos extrictamente positivos. Viene dada por:

\begin{equation}
x_{i}^{(\lambda)} =
\begin{cases}
\frac{x_{i}^{\lambda}-1}{\lambda} & \text { si } \lambda \neq 0 \\
\ln \left(x_{i}\right) & \text { si } \lambda=0
\end{cases}.
\end{equation}

En ambos casos, el parámetro $\lambda$ es estimado por máxima verosimilitud.

**Ejemplo**

La técnica de transformación por medio de potencias (power transform) permite estabilizar la varianza y ahcer que los datos se distribuyan de manera más similar a la distribución normal. Por lo general, se recomienda el uso de este tipo de transformaciones en datasets con pocas observaciones pues generalizan de manera rápida, además son fácilmente interpretables por medio del valor $\lambda$. Se definen distintas distribuciones de datos y se implementan las transformaciones anteriores

In [ ]:
from sklearn.preprocessing import PowerTransformer
n_sample = 1460

transformer_bc = PowerTransformer(method='box-cox')
transformer_yj = PowerTransformer(method='yeo-johnson')

Se genera un dataset con diferentes distribuciones por columna

In [ ]:
df = pd.DataFrame({
    'x_lognormal': np.random.lognormal(size = n_sample,),
    'x_chisq': np.random.chisquare(5, n_sample),
    'x_weibull': np.random.weibull(30, n_sample),
    'x_gaussian': np.random.normal(loc = 25, size = n_sample),
    'x_uniform': np.random.uniform(0, 1, n_sample)
})

Se observan las distribuciones univariadas para cada columna en la diagonal

In [ ]:
sns.pairplot(df, diag_kind='kde')

Se transforman los datos usando los objetos anteriores


In [ ]:
df_bc = transformer_bc.fit_transform(df)
df_bc = pd.DataFrame(df_bc, columns=df.columns)

df_yj = transformer_yj.fit_transform(df)
df_yj = pd.DataFrame(df_yj, columns=df.columns)

Se observan los resultados para el método de Box - Cox

In [ ]:
sns.pairplot(df_bc, diag_kind='kde')

Se observan los resultados para Yeo-Johnson

In [ ]:
sns.pairplot(df_yj, diag_kind='kde')

**Ejercicio**

1. Obtenga los valores de lambda para cada uno de los de métodos revisados.

2. El preprocesamiento por transformación de cuantiles es un método robusto que permite transformar una distribución de datos en una variable uniforme o normal. Permite el reducir el impacto de outliers. Investigue su formulación, ventajas y desventajas, aplique el transformer `QuantileTransformer` en los datos recientemente generados para observar su comportamiento.

#### Normalización

Otro método de transformación de datos es la normalización de estos. Esto conisite en un mapeo a la bola cerrada según una norma a elección. Para la norma 'l2' en el espacio euclidiano de 3 dimensiones se tiene el normalizador

\begin{equation}
\frac{x_{i}}{\sqrt{x_{i}^{2}+y_{i}^{2}+z_{i}^{2}}}
\end{equation}

**Ejemplo** 

Se estudia como opera este transformador de datos en una visualización:

In [ ]:
df = pd.DataFrame({
    'x': np.random.randint(-50, 50, 1460),
    'y': np.random.randint(-60, 60, 1460),
    'z': np.random.randint(-70, 70, 1460)
})

# Permite generar graficos 3d
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=[12,9])
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(df.x, df.y, df.z)

Se estudia el impacto de la transformación

In [ ]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer()
df_1 = scaler.fit_transform(df)
df_1 = pd.DataFrame(df_1, columns=df.columns)

Finalmente se visualiza

In [ ]:
fig = plt.figure(figsize=[12,9])
ax = plt.axes(projection='3d')

ax.scatter3D(df_1.x, df_1.y, df_1.z)

#### Codificación de varaibles categóricas

Para el manejo de adecuado de variables categóricas,  se recomienda expresar sus valores en función de códigos númericos. El transformer `OrdinalEncoder` permite transformar características categóricas en códigos enteros (números enteros)

**Ejemplo**

Se define una variable categórica y se preprocesa según `OrdinalEncoder`


In [ ]:
df = pd.read_csv('data/train.csv', index_col='Id')

# Esta variable posee las categorias 'RL', 'RM', 'C (all)', 'FV' y 'RH'
data = df['MSZoning'].copy()

Se inicializa el codificador

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

Se entrena el codificador

In [ ]:
# Se le da forma a los datos
X = data.values.reshape([-1,1])

# Se entrena el codificador
enc.fit(X)

Se obtienen las categorías

In [ ]:
cats = data.unique().reshape([-1,1])

print('Codigos:')
enc.transform(cats)

Se comprueban los códigos anteriores

In [ ]:
codes = pd.Series(enc.transform(X).flatten(), name = 'codes', index=data.index)
data_codes = pd.concat([data,codes], axis=1)

Se observa por ejemplo que el código 2, corresponde a 'RH'

In [ ]:
data_codes.groupby('codes').get_group(2).head()

#### Codificación por variables dummy

Un problema común con la códificación ordinal es que las variables pasan a ser consideradas continuas por algoritmos de machine learning (en especial por la API *estimators* de scikit-learn). Para evitar esto es posible convertir cada categoría en una columna por si sola y asignar un 1 cuando esté presente. 

Esto se puede llevar a cabo por medio del transformador ` OneHotEncoder`.

**Ejercicio**

1. Utilice este transformador en los datos categóricos anteriores. Los resultados serán entregados en formato *sparse* por  lo que tendrá que hacer uso del método `.toarray()` de los arreglos de NumPy.

2. Compare con la función `get_dummies` de pandas.

In [ ]:
import pandas as pd

#### Transformers definidos 

Si se desea realizar una transformación no estándar en los datos, es posible utilizar la clase `FunctionTransformer` y proporcionar una función de transformación.

**Ejemplo**

Se define un dataset y se opera con un transformador propio

In [ ]:
df = pd.DataFrame({
    'cat_1': np.random.randint(30,80,size = 5),
    'cat_2':range(5),
     })
df

Se define un transformador propio

In [ ]:
from sklearn.preprocessing import FunctionTransformer

def mapping(x):
    x.copy()
    x['cat_1'] += 5
    x['cat_2'] = x['cat_2'].apply(str) + '+ 1'
    
    return x

# se inicializa el transformador
trasformer = FunctionTransformer(mapping)

Se aplica a los datos anteriores

In [ ]:
trasformer.transform(df)

### Flujos de transformación con Pipelines y Compose

Las transformaciones en un dataset son combinadas entre si, hasta obtener una versión ordenada de los datos, posteriormente, estas se combinan con estimadores para formar un flujo de trabajo *input-output*. En Sckit-Learn el flujo antes nombrado de denomina *composite estimator* y se construye por medio de objetos tipo `Pipeline`.


Los objetos `Pipeline` se pueden utilizar para mezclar múltiples transformadores y estimadores generando un único tratamiento, su utilidad se expresa en la simplificación de etapas fijas en el tratamiento de datos, manteniendo un lenguaje sencillo e intuitivo. Se importan desde el módulo `pipeline` por medio de la convención

```python
from sklearn.pipeline import Pipeline
```

**Ejemplo**

Se utiliza una pipeline para transformar datos. En primera instancia se define un dataset por medio de las variables 'MSZoning' y 'LotArea' del dataset 'HousePricing'.

In [ ]:
df = pd.read_csv('data/train.csv',
                 usecols=['Id', 'MSZoning', 'LotArea'],
                 index_col='Id')

In [ ]:
df.plot.kde()

Sobre el dataset cargado, se efectúa una transformación ordinal sobre 'MSZoning', la variable 'LotArea' se escala según rango y se le aplica una transformación de Yeo-Johnson. El procedimiento mencionado, se lleva  a cabo mediante *pipelines*.

En primer lugar, las series de pandas, se consideran objetos con dimensión del tipo `(n,)` al traspasarlos a formato NumPy. Para poder trabajar con transformers de sklearn, sobre series de pandas, podemos transormar la dimensión de la serie por meido dl método `.reshape([-1,1])` para que pase a ser considerada como arreglo de NumPy de dimensión `(n,1)`. Para automtizar tal proceso, se define la función `data_1d`.

In [ ]:
def data_1d(df_column):
    return df_column.values.reshape([-1,1])

Luego se definen los transformadores que deseamos utilizar

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer

#reshape
rs = FunctionTransformer(data_1d)

#categorico
ordinal = OrdinalEncoder()

# Numericos
numeric_1 =  MinMaxScaler() #minmax
numeric_2 =  PowerTransformer(method='yeo-johnson') #yeo-johnson

Si se quisieran llevar a cabo los procesos de transformación, estos se harían de la forma:

In [ ]:
'''Caso categorico'''

data_cat = df.MSZoning.copy()
data_cat = rs.fit_transform(data_cat)
data_cat = ordinal.fit_transform(data_cat)
data_cat = pd.Series(data_cat.flatten(), name = 'MSZoning')
data_cat.head(3)

In [ ]:
'''Caso numerico'''

data_num = df.LotArea.copy()
data_num = rs.fit_transform(data_num)
data_num = numeric_1.fit_transform(data_num)
data_num = numeric_2.fit_transform(data_num)
data_num = pd.Series(data_num.flatten(), name = 'LotArea')
data_num.head(3)

La manera anterior es bastante clara de comprender, sin embargo, es redundante y repite muchos patrones de asiganción tediosos. Utilizando pipelines, es posible reducir la cantidad de código, sin reducir simpleza.

Para definir la *pipeline* se debe definir una lista de tuplas, donde cada una posee un identificador (*nombre definido por el usuario*) y una operación a realizar sobre el dataset. en el caso de la variable 'MSZoning' se puede definir según el siguiente esquema

In [ ]:
from sklearn.pipeline import Pipeline

cat_pipe = Pipeline([('reshape', rs), ('ordinal', OrdinalEncoder())])

Por otra parte, para la variable 'MSZoning'

In [ ]:
num_pipe = Pipeline([('reshape', rs), ('scaler', MinMaxScaler()),
                     ('yeo-johnson', PowerTransformer(method='yeo-johnson'))])

Los objetos tipo `Pipeline` permiten encadenar operaciones siguiendo el orden de la lsita de tuplas proporcionadas. En el caso de las pipelines proporcionadas, se efectúa primero (en ambas) la transformación `rs` que permite alterar la diemnsión de la series a operar. 

Si se desea solo obtener parámetros de transformación, se puede utilizar el método `.fit` sobre un objeto tipo `Pipeline`. Este actúa como un `map` aplicado a cada objeto transformador en cada tupla proporcioanada, de la misma forma, se pueden aplicar los métodos de transformación `transform` y de obteneción de parámetros con transformación directa `fit_transform`. 

En el caso anterior, esto corresponde a 

In [ ]:
cat_pipe.fit(df.MSZoning)
cat_pipe.transform(df.MSZoning)

# Alternativamente cat_pipe.fit_transform(df.MSZoning)

In [ ]:
num_pipe.fit(df.LotArea)
num_pipe.transform(df.LotArea)

# Alternaitvamente num_pipe.fit_transform

Se pueden utilizar los identificadores de cada paso (atributo `.steps`) para comprender las transformaciones de un objeto tipo `Pipeline`:

In [ ]:
[pipe[0] for pipe in num_pipe.steps]

En este caso, se observa que las transformaciones numéricas corresponden a un reshape, luego a escalar los datos y finalmente a transformarlos según el método de Yeo-Johnson.

**Ejercicio**

1. Genere un trasformador personalizado que permita transformar los datos de una pipeline diseñana sobre arreglos unidimensionales en series de Pandas. La función a implementar debe recibir un arreglo y un nombre de columna, debe entregar una serie con los datos transformados cuyo nombre es el nombre de la columna procesada. Añada este último transformador a las pipelines `cat_pipe` y `num_pipe`.

Es muy frecuente, es que los datos sea heterogéneos por ejemplo, es normal encontrar datasets con variables ordinales, categoricas, y numéricas. Para utilizar *pipelines* en este contexto, se necesitaria definir una por cada variable, repitiendo varios componentes de código entre variables que son del mismo tipo, esto resulta en una redundancia excesiva que se puede atacar por medio 
de objetos tipo `ColumnTransformer`, miembros del módulo `compose`. Estos objetos permite separar flujos de preprocesamiento, permitiendo seleccionar por columna o grupos de columna dentro de una *pipeline*.

**Ejemplo**

Se utiliza un objeto tipo `ColumnTransformer` para tratar datos heterogéneos. En primera ligar se importan las transformaciones a realizar

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

Para trabajar en este ejemplo, se cargará la base 'Titanic', esta consiste en una base pequeña con características de pasajeros del Titanic y una variable categórica 1-0 que indica si sobreviveron o no al choque hundimiento. Se carga la base

In [ ]:
data = pd.read_csv('data/titanic_train.csv', index_col='PassengerId')
data.head()

Separamos las columnas en numéricas y categóricas

In [ ]:
num_cols = ['Age','Fare']
cat_cols = ['Embarked','Sex','Pclass']

Ya sabemos llenar valores faltantes con `.fillna()`, una alternativa es el uso de objetos `SingleImputer` del módulo `impute`. Este tipo de objetos permiten llenar valores faltantes de acuerdo a un método de agregación sobre los valores con información completa en las columnas de un dataset. Se usará la estrategia de llenado 'constant' que es equivalente a `.fillna()` pues llena todos los datos faltantes con el mismo valor proporcionado. 

Se procede a inicializar el objeto `SimpleImputer`

In [ ]:
from sklearn.impute import SimpleImputer

Se generan pipelines de preprocesamiento sobre las variables categóricas. Esta pipeline cambia todos los valores faltantes por 'NA' y luego genera una codificación Dummy.

In [ ]:
cat_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('encoding', OneHotEncoder(handle_unknown='ignore'))
])

Para las variables numéricas se genera la siguiente pipeline. esta llena los valores faltantes usando la mediana y luego estandariza.

In [ ]:
num_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaling',StandardScaler())
])

Se combinan los procesos por medio de `ColumnTransformer`, este objeto se inicializa entregando una lista de tuplas, donde la primera componente es un identificador, la segunda una pipeline y la tercera un lista de strings contenedora de las columnas a tratar.

In [ ]:
prep = ColumnTransformer(
    transformers=[
        ('num', num_pipe, num_cols),
        ('cat', cat_pipe, cat_cols)])

Finalmente se aplican los procedimientos planificados en la variable `prep`

In [ ]:
prep.fit_transform(data)

Se obtiene los datos transformados en formato NumPy.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_test_split()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

**Ejercicio**

Las pipelines pueden manejar transformadores de todo tipo y conectarlos entre si, observe que `ColumnTransformer` es un transformador por si solo, luego puede conectarlo con otro objeto por medio de una nueva pipeline. En particular, se pueden conectar transformadores con estimadores, estos ultimos son contenedores de rutinas de aprendizaje automático, que se entrenan por medio del método (transversal en la API *estimators*)`.fit()`. El objetivo de este ejercicio es que comience a utilizar estimadores en pipelines, para ello:

1. Importe la clase `RandomForestClassifier` del submódulo `ensemble` de Scikit-Learn.
2. Importe la función `train_test_split` del submódulo `model_selection` de  Scikit-Learn.
3. Estudie la documentación de ambas clases, luego utilice `train_test_split` para generar conjuntos de entrenamiento y test, denominados `X_train`, `X_test`, `y_train`, `y_test`. En este caso la variable de respuesta `y` es la columna `'Survived'`. ¿Que porcentaje del conjunto de datos pasa a ser *test* por defecto?
4. Inicialice un clasificador de *bosque aleatorio* (Random Forest) con 10 estimadores.
5. Genere una pipeline que preprocese los datos según el esquema viston anteriormente y que como paso final entrene el clasificador Random Forest inicializado por usted. Esto se debe llavar a cabo por medio del método `.fit()`
6. Estudie el porcentaje de aciertos del clasificador en datos test por medio del método `.score()` aplicado a la pipeline producida.

**Obs**: Observe que para de pueden construir transformadores personalizados, haciendo uso de la clase `FunctionTransformer`. Si por otra parte se desea generar un transformador *desde 0* se puede crear una clase derivada de `BaseEstimator` y `TransformerMixin` (herencia múltiple, si). Para que la clase pueda actuar dentro de una pipeline, hace falta anular los métodos `fit`, `.transform` y `.fit_transform` (este último es opcional). Es un **buen** ejercicio generar un transformador usando este último método.

## Manejo de valores faltantes

Por lo general, existen razones prácticas y conceptuales a tener en cuenta cuando se trabaja con valores faltantes. 

En primer lugar, la falta de información introduce sesgos en los modelos de datos, pues hace que las muestras obtenidas no sean representativas del fenómeno que se desea estudiar, esto genera conclusiones sesgadas y puede llevar a tomar malas decisiones. 

En cuanto al componente práctico, los valores faltantes son incompatibles con algunos modelos de aprendizaje automático, debido a que estos modelos son parte de la razón fundamental de analizar un fenómeno por medio de datos,es que se necesita comprender bien los mecanismos de manejo de este tipo de valores. 

Según el contexto teórico anterior, el mecanismo de pérdida de información MCAR es el más sencillo en términos de modelación, pues solo requiere parametrizar la matriz indicadora de valores faltantes, sin considerar información fuera dentro de dataset. El test *MCAR de Little* sirve para probar si la información faltante en un dataset sigue la hipótesis MCAR. 

El test de Little evalúa diferencias en media entre subgrupos de datos con valores faltantes. Es una generalización del test-$t$ mencionado anteriormente. El estadístico de test es una suma ponderada según la ecuación:

\begin{equation}
\tag{eq:3}
\label{eq:3}
d^{2}=\sum_{j=1}^{J} n_{i}\left(\hat{\boldsymbol{\mu}}_{j}-\hat{\boldsymbol{\mu}}_{j}^{(\mathrm{ML})}\right)^{\mathrm{T}} \hat{\mathbf{\Sigma}}_{j}^{-1}\left(\hat{\boldsymbol{\mu}}_{j}-\hat{\boldsymbol{\mu}}_{j}^{(\mathrm{ML})}\right)
\end{equation}

Donde $n_j$ representa el numero de valores faltantes de la columna $j$. Dentro de la columna $j$ se generan grupos de valores faltantes en función de su relación con los valores faltantes de las demás columnas, así por ejemplo, el grupo 1 puede contabilizar solo aquellos valores faltantes para aquellas componentes (filas) presentes unicamente en la columna $j$ y que presentan información completa para todo $i \neq j$, por otra parte, el grupo 2, puede poseer aquellos valores faltantes para cuya componente posee información faltante en la columna $i \neq j$, pero inormación completa para todo $k \neq i,j$. Se generan grupos hasta agotar las combinaciones. Luego, $\hat{\boldsymbol{\mu}}_{j}$ representa un vector contenedor de medias para cada grupo, donde estas se medias, se calculan para las variables con información presente.
$\hat{\boldsymbol{\mu}}_{j}^{(\mathrm{ML})}$ representa un contenedor de estimadores de medias para cada grupo por medio de máxima verosimilitud. Finalmente $\hat{\mathbf{\Sigma}}_j$ represeta la matriz de covarianza entre cada grupo de la variable $j$.

**Ejercicio**

En el módulo `mcar` Se implementa el test de Little para cuantificar si un conjunto de datos posee un mecanismo de información faltante MCAR. Revise el código de la función `little_mcar`, comprenda su implementación.


#### Eliminación

Es el método más sencillo, se conoce tambien como 'list-wise deletion' y consiste en eliminar filas o columnas de un dataset que presenten datos faltantes. Se puede acceder a este tipo de tratamiento por medio de `.dropna()` objetos de Pandas.

Otra forma de eliminación de datos se conoce como 'pair-wise deletion' que consiste en generar subconjuntos de un dataset en función de sus patrones de perdida de información. (Vea la implementación del test de Little). Posteriormente, se conducen análisis separados por patrón de perdida de información y se llegan a distintos modelos para cada uno.

Ambos métodos son simples de implementar y asumen que el mecanismo de perdida de información en el dataset es del tipo MCAR. En casos distintos (MAR o MNAR) su uso es contraindicado. Se recomiendan cuando el patrón de perdida de información observada (por ejemplo por medio de `mssingno`) es claramente aleatorio, y si además las variables con información faltante son 'pocas' y con 'pocos' valores faltantes. La definición de 'poco' varia en función del problema, pero una buena huerística puede ser inferior al 15% en variables de poca importancia.

#### Imputación

Corresponde al llenado de información faltante por medio de estimaciones. Para efectuar este tipo de operaciones es importante tener en cuenta los mecanismos de pérdida de información latentes en los datos. Cabe destacar que el principal objetivo de la imputación no es maximizar la precisión (o maximizar metricas de similitud), sino que más bien, busca preservar las características del dataset inicial, generando uno con información completa que permita dilucidar las dinámicas implicadas en el fenómeno estudiado. 

Como resumen, los métodos de eliminación tienen más sentido bajo la hipótesis MCAR, luego si por ejemplo bajo el test de Little se adquiere un valor p > 0.05, cobra sentido evaluar tales estrategias.

##### Imputación singular

Como se ha venido aplicando a lo largo de este curso, corresponde a llenar valores faltantes con un valor único basado en la información observada, se requiere por tanto, tener evidencia de que el mécanismo de pérdida de información sigue la hipótesis MAR. 

Por lo general este tipo de imputación presenta un buen rendimiento empírico en tareas de ciencia de datos y es ampliamente recomendado, sin embargo, aplicar este tipo de métodos puede afectar el calculo de varianzas y covarianzas. 

En pandas podemos acceder a este tipo de imputación por medio del método `.fillna()` ya sea entregando un valor precalculado (media, mediana, moda, etc...) o utilizando los argumentos `ffill` y `bfill`. 


Existen directrices a tener en cuenta al momento de tratar valores faltantes:

1. Valores faltantes en variables categóricas / ordinales:
    * Transformar valores faltantes en una nueva categoría.
    * Utilizar códificación Dummy en variables categóricas
    * Agrear la categoría de valor faltante como orden inicial o final en categorias ordinales.


2. Valores faltantes continuos:
    * Probar métodos de imputación o eliminación.
    * En mecanismos MCAR, utilizar media / mediana.
    * En MAR o MNAR hay que tener en cuenta que la varianza no será representativa.

**Ejemplo**

Se desea aplicar el test de Little sobre el dataset 'HousePricing'. Para ello, se agrupan las variables de la base según el tipo de columna.

In [ ]:
df = pd.read_csv('data/train.csv', index_col='Id')

cat_cols = [
    'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood',
    'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
    'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
    'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive',
    'MiscFeature', 'SaleType', 'SaleCondition'
]

ordinal_cols = [
    'LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
    'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual',
    'GarageCond', 'PoolQC', 'Fence'
]

# Adquieren las categorias de cada variable
ordinal_cat = [['Reg', 'IR1', 'IR2', 'IR3'],
               ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'], ['Gtl', 'Mod', 'Sev'],
               ['Po', 'Fa', 'TA', 'Gd', 'Ex'], ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['NA', 'No', 'Mn', 'Av', 'Gd'],
               ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
               ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
               ['Po', 'Fa', 'TA', 'Gd', 'Ex'], ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
               ['NA', 'Po', 'Fa', 'TA', 'Gd',
                'Ex'], ['NA', 'Unf', 'RFn', 'Fin'],
               ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['NA', 'Fa', 'TA', 'Gd', 'Ex'],
               ['NA', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']]

num_cols = [
    'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
    'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
    'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
    'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
    'MoSold', 'YrSold'
]

Los valores perdidos en las variables categóricas serán completados con la nueva categoría 'NA', luego se procesan según una codificación Dummy.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Pipeline categorica
cat_pipe = Pipeline(
    steps=[('imputer_cat', SimpleImputer(strategy='constant', fill_value='missing')), 
           ('onehot',OneHotEncoder(sparse=False, handle_unknown='ignore'))])

Los datos numéricos serán estandarizados

In [ ]:
# Pipeline Numerica
num_pipe = Pipeline(steps=[('scaler', StandardScaler())])

Los valores ordinales serán tratados de la misma manera

In [ ]:
# Pipeline Ordinal
ord_pipe = Pipeline(
    steps=[('imputer_ord', SimpleImputer(strategy='constant', fill_value='NA')),
           ('ordinal', OrdinalEncoder(categories = ordinal_cat))])

Se componen las pipelines generadas

In [ ]:
#Preprocesador Compuesto
prep = ColumnTransformer(
    transformers=[('num', num_pipe, num_cols), 
                  ('cat', cat_pipe, cat_cols), 
                  ('ord', ord_pipe, ordinal_cols)])

Observe que bajo este tratamiento, las variables ordinales y categóricas tendrán una categoría extra que aportará información al modelo y que por tanto no será contabilizada por el test de Little.

Es **importante** que observe que el preprocesamiento fue realizado sobre las variables regresoras (no la dependiente), pues al momento de recibir nuevos datos, serán las columnas asociadas a variables regresoras,las que vendrán con información.

Se preparan los datos según las transformaciones anteriores.

In [ ]:
# Variables regresoras
X = df.drop('SalePrice', axis=1).copy()

# Variable dependiente
y = df['SalePrice'].copy()

# Se preparan los datos
X_prep = prep.fit_transform(X)

Observe que para las variables categóricas, se generan nuevas columnas asociadas a sus categorías. Se puede acceder a tales nuevas columnas por medio del atributo publico `.named_transformers_` del preprocesador

In [ ]:
# Se obtienen las variables categoricas transformadas
post_cat = prep.named_transformers_['cat'][-1]
cat_cols_fit = post_cat.get_feature_names(cat_cols)

Para las variables numéricas y ordinales, no hay cambios en las columnas.

In [ ]:
# columnas del datase luego de transformarlo
post_cols = list(num_cols) +list( cat_cols_fit) + list(ordinal_cols)
len(post_cols)

Se construye el dataset transformado

In [ ]:
df_post = pd.DataFrame(data=X_prep, columns=post_cols)

La dimension del nuevo dataset es:

In [ ]:
df_post.shape

Se Aplica el test de Little

In [ ]:
from mcar import little_mcar
little_mcar(df_post)

In [ ]:
import pandas as pd

In [ ]:
pd.

Con el valor $p$ obtenido se tiene que las diferencias en medias producidas por los patrones de datos perdidos, son **inconsistentes** con mecanismo MCAR. Por lo tanto, no se recomienda utilizar metodos de eliminación de valores faltantes, a menos que la la columna donde se encuentren estos valores sea de poca significancia en comparación a la variable dependiente.


**Ejercicios**

1. Compare la `get_dimmies` de pandas con el método one

2. El módulo `impute` de Scikit - Learn permite hacer imputaciones singulares por medio de la función (antes implementada) `SimpleImputer`. Debido a que el dataset 'HousePricing' con alta probabilidad no sigue un mecanismo de perdida de información MCAR, es posible asumir que sigue un patrón MAR (o MNAR). En este último caso, podemos asumir que los valores faltantes se pueden estimar por medio de los presentes.

    1. Utilice `SimpleImputer` para llenar los valores faltantes de 'LotFrontage' en el dataset `df_post`.

    2. Utilice el estimador `KNNImputer` para llenar los demás valores faltantes. **Obs**: Este método corresponde a llenar valores faltantes por medio de KNN, deberá elegir el número de vecinos para la imputación. 

Observe que en general puede utilizar estimadores para llenar valores faltantes, un ejemplo común es el uso de interpoladores. en términos generales, se puede utilizar un modelo sobre los datos con información completa para imputar valores faltantes. Estos métodos de imputación flexible pueden ser fácilmente implementados y se conoce como **métodos de imputación multivariada**. 

#### Imputación múltiple

Un problema de la imputación singular es que modela los datos como uno completo, sin considerar la incertidumbre inherente de los datos. Los métodos de imputación múltiple permite generar estadísticos insesgados para imputar y considerar incertidumbre en la imputación. Este método es el único lo suficientemente robusto para manejar datos faltantes fuera de la hipótesis MCAR. Su proceso de calculo se resume en:

1. Escoger un modelo de imputación uni-o multi -variado.
2. Generar $m$ datasets completos con los métodos elegidos anteriormente.
3. Analizar cada dataset generado aplicando la técnica de modelación que el problema requiere (regresión lineal, SVM, etc...) contra la variable de respuesta

4. Analizar los efectos de imputación de valores perdidos sobre los estimadores utilizados.


El objeto `IterativeImputer` del módulo `impute` permite utilizar el método de imputación múltiple basado en ecuaciones encadenadas [MICE](https://www.jstatsoft.org/article/view/v045i03). A diferencia del método antes descrito, retorna imputaciones singulares por variable uni- o muti- variadas. 

Su uso es experimental y se activa siguiendo la sintaxis:

```python
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
```

**Ejemplo**

Se utiliza el imputador iterativo para los estimadores de Random Forest y KNN.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

rf = RandomForestRegressor(random_state=0, n_estimators=5)
knn = KNeighborsRegressor(n_neighbors=8)

imputer_RF = IterativeImputer(estimator=rf,
                              skip_complete=True,
                              verbose=1,
                              random_state=1)

imputer_KNN = IterativeImputer(estimator=knn,
                              skip_complete=True,
                              verbose=1,
                              random_state=1)

In [ ]:
imputer_RF.fit_transform(df_post)

In [ ]:
imputer_KNN.fit_transform(df_post)

**Ejercicio** (Opcional)

1. Investigue sobre las reglas de Rubin para imputación múltiple. Utilice el esquema de cuatro pasos definido anteriormente y aplique las reglas para analizar la sensibilidad imputación - predicción.

## Detección y manejo de Anomalías

Una anomalía (*outlier* en ingles) es un dato significativamente distinto a los demás. Se puede considerar como una observación cuya desviación del conjunto de datos permite establecer la hipótesis, de que su generación fue obtenida por un mecanismo distinto al principal en la modelación de un fenómeno.

Las anomalías contienen por tanto información sobre características anormales de las entidades y esquemas que impactan el proceso generativo de los datos. Reconocer estas observaciones permite, desde el punto de vista teórico, mejorar el entendimiento de los problemas modelados. Desde el punto de vista práctico, permite mejorar procesos de adquisición de datos y presición de modelos. En este último capitulo, se estudian algunas técnicas de detección de anomalías.



### Métodos de manejo de anomalías

Se revisan algunas técnicas de detección de outliers revisando su formulación.

#### Desviación Estándar

Si se estima que la variable a estudiar e distribuye de manera normal, entonces el 95% de los datos se encuentra a 2 desviaciones estándar de la media, mientras que el 99.7% se encuentra dentro de 3 desviaciones estándar. Basándose en esto, se puede considerar que cualquier punto fuera de 3 desviaciones estándar de la media como candidato a anomalía. Una forma más flexible de estimar anomalías usando el principio de normalidad, es por medio de z-scores. 

**Ejemplo**

Se genera un dataset con valores anomalos y se estudian sus estadisticos z asociados

In [ ]:
from scipy.stats import zscore

data = np.random.normal(size=1000)
data[-5:] = [3.5,3.6,4,3.56,4.2]
data = pd.Series(data, name = 'test z')
data.head()

Se calculan los z-scores y se comparan con un valor umbral, en este caso 3. Se puede flexibilizar tal valor con tal de explorar distintos rangos de datos.

In [ ]:
data[np.abs(zscore(data)) > 3]

Si se va a estudiar una columna con ouliers mediante este método, es coveniente hacer un test de normalidad. Si la variable no cumple con la hipótesis, es posible preprocesarla usando el método de box-cox, Yeo-Johnson o Inter Quantilico. Se recomienda este último por ser robusto a outliers. 

**Obs**: para esta formulación, una anomalía es un punto *fuera de rango* si bien son definiciones similares, por lo general son distintas.

#### IQR: Rango intercuantílico

Si se desea analizar datos univariados que no siguen una distribución normal, se puede estudiar su rango intercuantílico.

**Ejemplo**

Se genera un dataset con outliers y se estudia este método.

In [ ]:
from scipy.stats import iqr
data = np.random.beta(2,3,size=10000)
data[-5:]=[-0.3,1,1.2,0.9,2]

data = pd.Series(data, name = 'test iqr')

Se calcula el rango intercuantílico y se definen cotas superiores e inferiores en función de su valor.

In [ ]:
iqr_val = iqr(data)

cota_inf = np.percentile(data,25) - iqr_val*1.5
cota_sup = np.percentile(data,75) + iqr_val*1.5

Se estudian los datos fuera de rango

In [ ]:
data[data < cota_inf]

In [ ]:
data[data > cota_sup]

Una manera visual de comprobar este método (y más rápida) es por medio de gráficos de caja.

In [ ]:
fig, ax = plt.subplots(figsize = [6,5])
data.plot.box(ax = ax)

el rango intercuantílico se utiliza para medir la dispersión de los datos, esto se logra separandolos en cuartiles (cuatro intervalos). La diferencia entre el primer cuartil y el tercero es el IQR (=Q3 - Q1). En el gráfico de caja, vemos que los outliers están sobre y bajo las lineas rectas, cada una representa Q1 - 1.5 x IQR (linea inferior) y Q3 + 1.5 x IQR (linea superior) Los valores dentro de la caja corresponden al IQR y la linea central es la mediana de los datos.

### Modelo de mezcla de gaussianas

El modelo de mezcla de Gaussianas corresponde a un modelo para clusterización de muestras que obedece un enfoque generativo. Este modelo puede ser pensado como una generalización del modelo K-Means ya que permite que los clusters encontrados tengan forma anisotrópica. Este modelo permite mayor flexibilidad. Se estudia su formulación.

Sea $X = \left \{ x_i \right \}_{i=1}^N \subset \mathbb{R}^L $. El modelo de mezcla Gaussiano tiene como supuesto principal el hecho que los datos observados son muestras *i.i.d* de la distribución $p(x)$, la cual tiene la siguiente forma:

\begin{equation}
p(x) = \sum_{k=1}^K \pi_k \mathcal{N}(x|\mu_k, \Sigma_k)
\end{equation}

como se observa la distribución $p$ depende de varios parámetros y consiste en un modelo ideado para aproximar distribuciones como la suma de $K \in \mathbb{N}$.

- El hiper parámetro $K \in \mathbb{N}$ corresponde a la cantidad de clusters, este debe ser predefinido.
- $\pi_k \geq 0$ corresponde a la proporción de mezcla asociada la cluster $k$, se cumple que $\sum_{k=1}^K \pi_k = 1$.
- $\mu_k$ es la media de la Gaussiana asociada al cluster $k$.
- $\Sigma_k$ es la matriz de covarianza de la Gaussiana asociada al cluster $k$.

Adicionalmente la variable latente $z = (z_1,\ldots,z_K) \in \{ 0,1 \}^K$ indica a cual de los $K$ clusters pertenece la observación $x$. Si $z_k = 1$ entonces $x$ pertenece al cluster $k$, solamente 1 de los elementos de $z$ es distinto a 0.

La probabilidad de que una muestra pertenezca al cluster $k$ puede escribirse como:

\begin{equation}
p(z_k = 1) = \pi_k
\end{equation}

por lo tanto la probabilidad conjunta de $z$ es:

\begin{equation}
p(z) = \prod_{k=1}^K \pi_k^{z_k}
\end{equation}

Por otro lado se puede escribir la probabilidad condicional de $x$ dado que pertenece al cluster $k$:

\begin{equation}
p(x|z_k = 1) = \mathcal{N}(x|\mu_k, \Sigma_k)
\end{equation}

En consecuencia la distribución condicional de $x$ dado $z$ es:

\begin{equation}
p(x|z) = \mathcal{N}(x|\mu_k, \Sigma_k)^{z_k}
\end{equation}

Se procede a calcular la distribución posterior de $z_k$:

\begin{equation}
p(z_k = 1 | x) = \frac{p(x|z_k = 1)p(z_k = 1)}{p(x)} = \frac{\pi_k \mathcal{N}(x|\mu_k, \Sigma_k)}{\sum_{k=1}^K \pi_k \mathcal{N}(x|\mu_k, \Sigma_k)}
\end{equation}

este termino se conoce como responsabilidad de la componente $k$ por la observación $x$. Se define $\gamma_{z_k}(x) = p(z_k = 1 | x)$.

Con una expresión para la distribución posterior y recordando el hecho que las muestras $X$ son *i.i.d*, se procede a calcula la log-verosimilitud de la muestra:

\begin{equation}
\log p(x_1, \ldots, x_N) = \sum_{n=1}^N \log p(x_n) = \sum_{n=1}^N \log \sum_{k=1}^K \pi_k \mathcal{N}(x_n, \mu_k, \Sigma_k)
\end{equation}

La forma de esta verosimilitud no es tán simple de optimizar, se presentan complicaciones para manejar la suma dentro del logaritmo, por esta razón se utilizará un enfoque alternativo a máxima verosimilitud.

El algoritmo que suele ser usado para estimar los parámetros de la posterior es conocido como algoritmo EM (Expectation maximization), sus etapas se exponen a continuación:

- **Inicialización:** Se inicializan los parámetros $\mu_k^0$, $\Sigma_k^0$ y $\pi_k^0$ $\forall k =1,\ldots,K$ ; medias, matrices de covarinza y porciones de mezcla respectivamente. Luego se evalua el valor de la log-verosimilitud.


- **Paso E:** Se evaluan los terminos de responsabilidad utilizando los parámetros actuales

$$ \gamma_{z_{nk}}(x_n) = \frac{\pi_k \mathcal{N}(x|\mu_k, \Sigma_k)}{\sum_{k=1}^K \pi_k \mathcal{N}(x_n|\mu_k, \Sigma_k)}$$


- **Paso M:** Se re estiman los parámetros del modelo con los nuevos valores de responsabilidad encontrados utilizando las siguientes expresiones (condiciones de primer orden para optimalidad):

\begin{equation}
\mu_k = \frac{\sum_{n=1}^N \gammạ_{z_{nk}}(x_n)x_n}{\sum_{n=1}^N \gammạ_{z_{nk}}(x_n)}
\end{equation}

\begin{equation}
\Sigma_k = \frac{\sum_{n=1}^N \gammạ_{z_{nk}}(x_n)(x_n-\mu_k)(x_n-\mu_k)^T}{\sum_{n=1}^N \gammạ_{z_{nk}}(x_n)}
\end{equation}

\begin{equation}
\pi_k = \frac{1}{N} \sum_{n=1}^N \gammạ_{z_{nk}}(x_n)
\end{equation}


- **Evaluación:** Evaluar la log-verosimilitud con los nuevos parámetros. Si no hay convergencia (necesario definir un criterio de convergencia) repetir desde el **paso E**.

**Ejemplo**

Se procede a estudiar una aplicación del modelo

In [ ]:
n_samples = 500

# Datos aleatorios
np.random.seed(0)

C = np.array([[0., -0.1], [1.7, .4]])
C2 = np.array([[1., -0.1], [2.7, .2]])

X = np.r_[np.dot(np.random.randn(n_samples, 2), C),np.dot(np.random.randn(n_samples, 2), C2)]

Se agregan puntos fuera de rango

In [ ]:
X[-5:] = [[4,-1],[4.1,-1.1],[3.9,-1],[4.0,-1.2],[4.0,-1.3]]

Se grafican los datos

In [ ]:
fig = plt.figure(figsize=[6,5])
plt.scatter(X[:,0], X[:,1], alpha=0.5)

Se importa el modelo y se aplica

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=2)
gmm.fit(X)

Se utilizan las asignaciones de cluster para analizar los datos

In [ ]:
fig = plt.figure(figsize=[6,5])
pred = gmm.predict(X)

plt.scatter(X[:,0], X[:,1],c=pred, alpha=0.5)

Por lo general, para seleccionar el número de componentes de la mezcla, es necesario tener conocimiento del problema de fondo 

**Ejercicio**

1. Ejecute el modelo de mezcla de gaussianas para 3 componentes. Extraiga los potenciales outliers

### Modelos basados en densidad

Estos modelos aproximan la distribución de los datos a analizar asinando densidades (paremétricas o no paramétricas) a sus puntos. Obtienendo tal estimador, se puede tener una idea de la probabilidad de que un punto del dataset sea un outlier

### KDE: Estimador de densidad por kernel

Si $(x_1, \ldots, x_n)$  corresponden a observaciones univariadas iid obtenidas de alguna distribución desconocida. Entonces se puede estimar la densidad $f$ asociada a tal distribución por medio de $\hat{f}_{h}$ dada por:

\begin{equation}
\hat{f}_{h}(x =\frac{1}{n h} \sum_{i=1}^{n} K\left(\frac{x-x_{i}}{h}\right)
\end{equation}

Donde $K$ es una función no negativa conocida como *kernel* y $h >0$ es un parámetro de suavidad denotado como *ancho de banda*.

**Obs**: La definición aquí presentada hace referencia a kernels invariantes a reotación como el kernel gaussiano o RBF, en particular, la elección del kernel puede ser cualquier función no negativa y simétrica que tenga sentido como función base.

**Ejemplo**

A continuación se explora el método KDE para la extracción de outliers. Primero, se genera un dataset y luego se explora

In [ ]:
np.random.seed(7)

n = 1460
d = 3

eje = 6

X = np.zeros((n, d))

X[:int(n/2),0] = eje
X[(-int(n/2)+1):,0] = -eje

X += np.random.normal(0,1, size = X.shape)

sample = np.random.randint(n, size = int(n/100)) 

X[sample] +=  0.3*np.random.normal(0,5, size = (int(n/100),d))

Se procede a importar el método y a aplicarlo

In [ ]:
from scipy.stats.kde import gaussian_kde

kde = gaussian_kde(dataset=X.T)

scores = kde.evaluate(X.T)
idx = scores.argsort()
scores.sort()

Se observan los resultados

In [ ]:
fig = plt.figure(figsize=[6,5])

plt.bar(range(50),scores[:50])
plt.title('Outlier score')

In [ ]:
print('Cantidad de potenciales outliers:',len(sample))

Dado que el estimador kde estima la densidad asociada a la distribución de los datos, es posible asignar un puntaje a cada dato basándose en cuan baja es su densidad asociada por medio del kde entrenado. En el caso anterior se muestran los 50 puntajes más bajos. El arreglo de datos asociados a tales puntajes está en la variable `idx`. Con lo anterior, es posible seleccionar varaibles de acuerdo a su puntaje y relación con la columna de respuesta a modelar.

Se puede obtener una visualización del fenómeno anterior

In [ ]:
from mpl_toolkits import mplot3d

fig = plt.figure(figsize=[8, 5])
ax = plt.axes(projection='3d')

ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=scores, linewidth=0.5, alpha=0.6)

**Ejercicio**

1. Investigue que kernel es usado por defecto en el ejemplo anterior.

### Estimación de densidad por medio de KNN

Una manera sencilla de estimar densidades (y de manera no paramétrica) es por medio del método KNN. En esta caso, la densidad asociada a un punto $x$ vienen dada por $1/\bar{N_k}(x)$ donde $\bar{N_k}(x)$ es el promedio de los vecinos para $x$.

**Ejemplo**

Se inicializa el algoritmo para trabajar con 100 vecinos, luego se entrena en los datos

In [ ]:
from sklearn.neighbors import NearestNeighbors

K=100
knn = NearestNeighbors(n_neighbors=K)
knn.fit(X)

Para cada punto de `X` encontramos sus `K` vecinos mas cercanos, se almacenan en un arreglo de $K \times n$.

In [ ]:
D, _ = knn.kneighbors(X,K)

La densidad asociada a un punto sera el inverso del promedio de sus vecinos. Se calcula tal densidad para todos los puntos

In [ ]:
knn_density_est = K/D.sum(axis=1)

Se visualiza el experimento

In [ ]:
fig, ax = plt.subplots(figsize=[6,5])

scores = pd.DataFrame(data = knn_density_est, index = range(n))
scores.sort_values(by=0, inplace=True)

plt.title('Outlier score')
scores[:25].plot.bar(ax = ax)

In [ ]:
fig = plt.figure(figsize=[8, 5])

ax = plt.axes(projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2], c=knn_density, cmap='viridis', linewidth=0.5);


### Clustering DBScan 

DBScan: Density based spatial clustering of applications with noise, es un algoritmo de clustering ampliamente utilizado. Una de sus principales fortalezas es que depende de solo 2 hiper-parámetros y no requiere definir una cantidad de clusters de antemano.

Como su nombre lo indica este es un algoritmo que se apoya en nociones espaciales, por lo tanto requiere de una métrica para comparar la distancia entre puntos. En la mayoria de las aplicaicones se suele usar la norma euclidiana para estos propositos.

Los parámetros del modelo son:

- Radio de vecindad $\epsilon$
- Cantidad mínima de puntos en la vecindad $N_{min}$

para el resto del extracto estos se consideraran dados y fijos.

Sea $X = \left \{ x_i \right \}_{i=1}^N \subset \mathbb{R}^M $ una colección de puntos. El algoritmo DBSCAN se apoya sobre 3 nociones, que dependen de  las cuales se definen a continuación:

- **$\epsilon$-vecindad:** Sea $x \in X$, se define el conjunto
        
    $N_{\epsilon}(x) = \left \{ y \in X \hspace{2mm}| \hspace{2mm} d(x,y) \leq \epsilon \right \} $
    
    

- **Directamente denso-alcanzable:** El punto $x$ se dirá denso-alcanzable respecto al punto $y$ si cumple con las siguientes condiciones

    1) $x \in N_{\epsilon}(y)$
    
    2) $ | N_{\epsilon}(y) | \geq N_{min}$ (*Condición de punto nucleo*)
    
    
    
- **Denso-alcanzable:** El punto $x$ se dirá denso-alcanzable respecto al punto $z$ si existe una cadena de puntos $x_1,\dots, x_n \subset X$; donde $x_1= x$,  $x_n = z$ y tal que $x_{i+1}$ es directamente denso-alcanzable respecto a $x_i$ para $i=1,\dots,n-1$

Esta última noción viene a extender la defición de directamente denso-alcanzable. La propiedad declara que 2 puntos están conectados si existe una cadena de puntos cada uno de ellos con al menos $N_{min}$ puntos a una distancia menor o igual a $\epsilon$. Si 2 puntos son denso-alcanzables entre si, diremos que están **denso-conectados**.

- **Cluster:** Diremos que un conjunto no vacio $C \subset X$ es un cluster de $X$ si satisface las siguientes condiciones

    1) $\forall x,y \in X$, si $x \in C$ e $y$ es denso-alcanzable desde $x$, entonces $y \in C$. (maximalidad)
    
    2) $\forall x,y \in X$, entonces $x$ es denso-alcanzable desde $y$. (Conectividad)
    
    
Esta definición nos dice que un Cluster en el sentido de DBSCAN corresponde a un conjunto en el que todos sus elementos se relacionan entre si en el sentido de ser denso-alcanzables. Por otro lado, este conjunto no puede crecer más, ya que contiene todos los puntos alcanzables en principio (maximalidad).

En este contexto DBSCAN separa los puntos en $X$ en 3 categorías:

- **Puntos núcleos:** Corresponden a los puntos que poseen al menos $N_{min}$ "$\epsilon$-vecinos" ($ | N_{\epsilon}(y) | \geq N_{min}$), estos puntos pueden pensarse como ubicados en el interior de un cluster.

- **Puntos borde:** Corresponden a los puntos que son alcanzables desde un punto núcleo, pero $ | N_{\epsilon}(y) | < N_{min}$, corresponden a los puntos en el margen de un cluster.

- **Outliers:** Estos son los puntos que no son alcanzable por ningún otro punto, se les denota tambien como *puntos ruido*.

Dicho esto, el algoritmo DBSCAN opera recorriendo cada $x \in X$, determina cuales puntos $y \in X$ son denso-alcanzables desde el punto actual. Si $x$ es un **punto núcleo** se genera un cluster, por otro lado si $x$ resulta ser un **punto borde** no existirán otros puntos en $X$ que sean alcanzables desde $x$. Luego DBSCAN pasa a revisar el siguiente punto en $X$ hasta completar el recorrido de todos los elementos.

**Ejemplo**

Se inicializa el algoritmo DBScan

In [ ]:
from sklearn.cluster import DBSCAN

outlier_detection = DBSCAN(min_samples = 2, eps = 0.5)

Los parámetros `min_samples` indica el número mínimo de puntos núcleo, `eps` es la minima distancia para considerar a dos puntos en el mismo cluster.

Se procede a entrenar y obtiener las asignaciones de cada punto

In [ ]:
clusters = outlier_detection.fit_predict(X)

el número total de outliers viene dado por aquellos puntos etiquetados como '-1'. En este caso, se tienen 

In [ ]:
list(clusters).count(-1)

Se visualizan los resultados

In [ ]:
fig = plt.figure(figsize=[8, 5])

ax = plt.axes(projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2], c=clusters, cmap='viridis', linewidth=0.5);

**Ejercicio**

1. Utilice el clustering anterior para extraer los posible puntos anómalos de la base estudiada.

#### Bosque de aislamiento

Los métodos basdos en bosques aleatorios permiten trabajar con datos de alta dimensionalidad de manera sencilla. El bosque de aislamiento *Isolation Forest*, consiste en un estimador de puntajes anomalías. Para lograr tal cometido, el algoritmo actua de la siguiente manera:

1. Selecciona una variable $x_j$ al azar.
2. Selecciona un valor $x^{*}$ al azar entre $(\min(x_j),\max(x_j))$.
3. Basado en  $x^{*}$, genera una partición del dataset según las filas cuyo valor de la variable $x_j$ es mayor a $x^{*}$ (y respectivamente menor que $x^{*}$). 
4. Vuelve al paso 1 y aplicando sobre cada dataset inducido por la partición. 

Este tipo de partición recurrente genera un árbol, donde sus hojas vienen dadas por valores aislados. La cantidad de particiones necesarias para aislar tales puntos viene dada por su distancia con la raiz del árbol. Si tal distancia es pequeña, se espera que el valor aislado sea un outlier, para que esta afirmación tenga sentido estadístico, es necesario generar varios estimadores de aislamiento, dando lugar a un bosque (conjunto de arboles) de aislamiento.

**Ejemplo**

Se inicializa el algoritmo y se prueba en los datos anteriores.

In [ ]:
from sklearn.ensemble import IsolationForest
rfi = IsolationForest(max_samples=1460,
                      random_state=1)

El resultado es una clasificación entre puntos normales (1) y anomalías (-1).

In [ ]:
clas = rfi.fit_predict(X)

Se observa que al proporcionar todo el dataset, entrega un buen aproximado a los outliers introducidos:

In [ ]:
list(clas).count(-1)

En la visualización se aprecia el resultado

In [ ]:
fig = plt.figure(figsize=[8, 5])

ax = plt.axes(projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2], c=clas, cmap='viridis', linewidth=0.5);

**Ejercicio**

Los métodos de sobre elíptico (elliptic envelope),  Local Outlier factor (LOF) y one class support vector machines son bastante eficientes en la detección de outliers. Estudie los objetos:

* `neighbors.LocalOutlierFactor`
* `covariance.EllipticEnvelope`
* `svm.OneClassSVM`



1. ¿Como permiten (cada uno) la detección de outliers? 
2. Discuta las ventajas y desventajas de cada uno.
3. Entrene y aplique cada uno de estos métodos.
4. Estudie la formaluación matemática de cada uno de estos métodos.